## word2vec으로 추천 시스템 구현 

1. word2vec으로 트레이닝
2. itembased로 개인화 추천 

In [1]:
#def dataLoad():

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as mpl
from gensim.models import Word2Vec
from collections import defaultdict
%matplotlib inline

In [12]:
## string to int with ignore none disit
def toint(s,default=0):
    converted = default
    try :
        converted = int(s)
    except:
        #print('string convert exception "{0}" (default={1})"'.format(s , default))
        pass
    return converted

class music_recsys:
    model = None
    home_dir = '/Users/goodvc/Data/fastcampus/week5/resource/music-ds/'
    nn_func = None
    
    ## initialize
    def __init__(self):
        #self.data_load()
        #self.data_processing();
        print('init')
    
    ## dataset load
    def data_load(self):
        ## music streaming log data
        self.streamings = pd.read_csv(self.home_dir+'streaming_ds.csv', 
                                      names = ['MEM_UNO', 'SONG_ID', 'ARTIST_ID', 'GENRE_CODE', 'REG_DATE'])
        self.streamings.index = self.streamings.SONG_ID
        self.streamings = self.streamings.convert_objects(convert_numeric=True).dropna()
        self.genre = pd.read_csv(self.home_dir+'genre.csv', 
                                      names = ['GENRE_CODE', 'GENRE_NAME'], index_col = 0)
        self.artist = pd.read_csv(self.home_dir+'artist.csv', 
                                      names = ['ARTIST_ID', 'ARTIST_NAME'], index_col = 0)
    ## make corpus
    def make_sentences(self):
        corpus = []
        grouped = self.streamings.groupby(['MEM_UNO'])
        for idx, user in grouped:
            l = user['SONG_ID'].apply(lambda x: str(x)).values.tolist()
            if len(l)>2:
                corpus.append(l)
        return corpus
    
    def training(self):
        if self.model == None :
            self.scentences = self.make_sentences()
            self.model = Word2Vec(self.scentences, min_count = 5, size = 100, window = 5)
            print('model trained')
    
    def load_model(self, model_path):
        self.model = Word2Vec.load(model_path)
        print('model loaded', model_path)
        
    def save_model(self, model_path):
        self.model.save(model_path)        
   
    def nearest_neighbors(self, song_id, topn=10):
        if str == type(song_id):
            song_id=[song_id]
        result = []
        try:
            result = self.model.most_similar(positive=song_id, topn=topn)
        except:
            pass
        return result 
    def user_interests(self, uno):
        return self.streamings[self.streamings.MEM_UNO==uno].index.tolist()
    
    def cal_score(self, sim, pre_score):
        return  sim if pre_score < sim else pre_score
        #return  sim + pre_score
    
    def recommend_music(self, uno, topn=10):
        musics = defaultdict(float)
        interests = self.user_interests(uno)
        for song in interests:
            neighbors = self.nearest_neighbors(str(song))
            for (song2, sim) in neighbors:
                #if song2 in interests:
                #    continue
                pre = musics.get(song2, 0.0)
                musics[song2] = self.cal_score(sim, pre)

        return {'recommended' : sorted(musics.items(), key=lambda x: x[1], reverse=True)[:topn]
                ,'seens':interests}

    def print_artist(self, song_list ):
        for song in song_list:
            try:
                if type(song) in { tuple, list } :
                    song = song[0]
                song = int(song)
                artist = self.streamings.loc[song][:1].ARTIST_ID.tolist()[0]
                n = self.artist.loc[str(artist)][:1].ARTIST_NAME
                print(song, n)
            except:
                print(song, 'no-artist')

In [13]:
rs = music_recsys()
rs.data_load()

init


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pandas/io/parsers.py:1159: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
rs.training()

init
model trained


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pandas/io/parsers.py:1159: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [11]:
rs.save_model('./resource/music2vec_rs.model')

In [14]:
rs.load_model('./resource/music2vec_rs.model')

model loaded ./resource/music2vec_rs.model


In [16]:
## 84699045 원더걸스 (Wonder Girls) 유사한 음원은?
ret = rs.nearest_neighbors('84699045', topn=20)
rs.print_artist([84699045]+ret)

84699045 원더걸스 (Wonder Girls)
84581811 no-artist
84686902 BIGBANG
84760929 Simon Dominic
84568778 Apink (에이핑크)
84688028 스컬&하하
84618059 no-artist
84688029 스컬&하하
84702220 no-artist
84564465 구하라
84672668 유승우
84686922 원더걸스 (Wonder Girls)
84715237 김그림
84702304 티아라 (T-ara)
84581594 Patrick Joseph
84688715 틴 탑 (Teen Top)
84699077 원더걸스 (Wonder Girls)
84699078 원더걸스 (Wonder Girls)
84686921 원더걸스 (Wonder Girls)
84699047 원더걸스 (Wonder Girls)
84649923 딕펑스 (Dickpunks)


In [15]:
ret = rs.recommend_music(301863475 ,topn=20)
print('*** 추천 음악 ***')
rs.print_artist(ret['recommended'])
print('*** 시청한 음악 ***')
rs.print_artist(ret['seens'])

*** 추천 음악 ***
48197709 엠씨더맥스 (M.C the MAX)
16178434 SG워너비
16128654 이기찬
16131081 윤미래 (T)
16098763 뱅크
16174214 유엔 (UN)
52802787 프리스타일 (Free Style)
75195354 플라이 투 더 스카이 (Fly To The Sky)
80696497 씨야 (SeeYa)
80743667 플라워 (Flower)
44366571 윤민수 (Vibe)
16110221 컨츄리 꼬꼬
16108396 이수영
82030328 The 1975
16109561 크라잉 넛
81360096 에피톤 프로젝트 (Epitone Project)
34987791 노블레스 (Noblesse)
16128471 박지윤
82795971 Sam Smith
16186783 배치기
*** 시청한 음악 ***
15965255 no-artist
16186222 테이 (Tei)
17465599 버즈 (Buzz)
40207122 하동균
49619416 Boyz II Men
82872166 비스트 (Beast)
83360622 제아 (브라운아이드걸스)
